In [1]:
# check the pytorch installation
import torch
print(torch.cuda.is_available())       # Should be True
print(torch.backends.cudnn.enabled)    # Should be True

True
True


In [2]:
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import os
import csv
import numpy as np
import pickle
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import cv2

In [3]:
# camera-ready
import sys

sys.path.append("deeplabv3")
sys.path.append("deeplabv3/model")
from deeplabv3 import DeepLabV3

sys.path.append("deeplabv3/utils")
from utils import label_img_to_color


In [4]:
work_dir = '/home/kywch/deeplabv3/'
lib_dir = work_dir + 'deeplabv3/'

# Specify your output directory
output_dir=work_dir+"output/"

In [5]:
network = DeepLabV3("eval_val", project_dir="deeplabv3", 
                    pretrained_file=lib_dir+"resnet18-5c106cde.pth").cuda()

pretrained resnet, 18


In [6]:
network.load_state_dict(torch.load(lib_dir+"model_13_2_2_2_epoch_580.pth"))

<All keys matched successfully>

In [7]:
xxx = os.listdir('input')
print(xxx)

['street2.jpg', 'street1.jpg']


In [8]:
class DatasetVal(torch.utils.data.Dataset):
    def __init__(self, input_path, output_path):
        self.img_dir = input_path
        self.img_h = 1024
        self.img_w = 2048
        self.output_dir = output_path
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        
        self.examples = []
        file_names = os.listdir(self.img_dir)
        for file_name in file_names:
            if file_name.lower().endswith(('.jpg', '.png', '.jpeg')):
                img_id = file_name.split('.')[0]
                img_path = self.img_dir + '/' + file_name
                example = {}
                example["img_path"] = img_path
                example["img_id"] = img_id
                self.examples.append(example)
        #print(self.examples)

        self.num_examples = len(self.examples)
        print(str(self.num_examples) + " images were identified from the input directory.")
        
    def __getitem__(self, index):
        example = self.examples[index]

        img_id = example["img_id"]
        img_path = example["img_path"]
        img = cv2.imread(img_path, -1) # (shape: (1024, 2048, 3))

        # normalize the img (with the mean and std for the pretrained ResNet):
        img = img/255.0
        img = img - np.array([0.485, 0.456, 0.406])
        img = img/np.array([0.229, 0.224, 0.225]) # (shape: (512, 1024, 3))
        img = np.transpose(img, (2, 0, 1)) # (shape: (3, 512, 1024))
        img = img.astype(np.float32)

        # convert numpy -> torch:
        img = torch.from_numpy(img) # (shape: (3, 512, 1024))
        #label_img = torch.from_numpy(label_img) # (shape: (512, 1024))
        print(img_id)
        return (img, img_id)

    def __len__(self):
        return self.num_examples


In [9]:

val_dataset = DatasetVal(input_path=work_dir+"input", output_path=output_dir)

2 images were identified from the input directory.


In [10]:
val_dataset.examples

[{'img_path': '/home/kywch/deeplabv3/input/street2.jpg', 'img_id': 'street2'},
 {'img_path': '/home/kywch/deeplabv3/input/street1.jpg', 'img_id': 'street1'}]

In [11]:
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                         batch_size=1, shuffle=False,
                                         num_workers=1)

In [12]:
network.eval()

DeepLabV3(
  (resnet): ResNet_BasicBlock_OS8(
    (resnet): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d

In [13]:
count = 0

#produce labeled image
for step, (imgs, img_ids) in enumerate(val_loader):
    #print(img_ids)
    with torch.no_grad(): # (corresponds to setting volatile=True in all variables, this is done during inference to reduce memory consumption)
        imgs = Variable(imgs).cuda() # (shape: (batch_size, 3, img_h, img_w))
        outputs = network(imgs) # (shape: (batch_size, num_classes, img_h, img_w))
  
        ########################################################################
        # save data for visualization:
        ########################################################################
        
        outputs = outputs.data.cpu().numpy() # (shape: (batch_size, num_classes, img_h, img_w))
        pred_label_imgs = np.argmax(outputs, axis=1) # (shape: (batch_size, img_h, img_w))
        pred_label_imgs = pred_label_imgs.astype(np.uint8)

        for i in range(pred_label_imgs.shape[0]):
            if i == 0:
                pred_label_img = pred_label_imgs[i] # (shape: (img_h, img_w))
                img_id = img_ids[i]
                img = imgs[i] # (shape: (3, img_h, img_w))

                img = img.data.cpu().numpy()
                img = np.transpose(img, (1, 2, 0)) # (shape: (img_h, img_w, 3))
                img = img*np.array([0.229, 0.224, 0.225])
                img = img + np.array([0.485, 0.456, 0.406])
                img = img*255.0
                img = img.astype(np.uint8)

                pred_label_img_color = label_img_to_color(pred_label_img)
                overlayed_img = 0.35*img + 0.65*pred_label_img_color
                overlayed_img = overlayed_img.astype(np.uint8)
                #producing original predicted images
                cv2.imwrite(val_dataset.output_dir + "/" + img_id + "_pred_label.png", pred_label_img )
                
                # produce debugging info for the first 10 images
                if count < 10:
                    #producing colored predicted images
                    cv2.imwrite(val_dataset.output_dir + "/" + img_id + "_pred_label_colorimg.png", pred_label_img_color )
                    #producing overlayed images
                    cv2.imwrite(val_dataset.output_dir + "/" + img_id + "_overlayed.png", overlayed_img)
                

        ########################################################################
        # quick counting
        ########################################################################
        count += 1
                

street2
street1


/home/kywch/.conda/envs/rcctorch-1.2/lib/python3.7/site-packages/torch/nn/functional.py:2390: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/home/kywch/.conda/envs/rcctorch-1.2/lib/python3.7/site-packages/torch/nn/functional.py:2479: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [14]:
# generate the summary csv file from the labeled files
import glob
output_files = glob.glob(val_dataset.output_dir + "/*pred_label.png")

In [15]:
print(val_dataset.output_dir)
print(output_files)

/home/kywch/deeplabv3/output/
['/home/kywch/deeplabv3/output/street2_pred_label.png', '/home/kywch/deeplabv3/output/street1_pred_label.png']


In [16]:
# deeplabv3 - cityscape labels

class_names = ['pcnt_road', 'pcnt_sidewalk', 'pcnt_building', 'pcnt_wall', 'pcnt_fence',
    'pcnt_pole', 'pcnt_traffic_light', 'pcnt_traffic_sign', 'pcnt_vegetation', 'pcnt_terrain',
    'pcnt_sky', 'pcnt_person', 'pcnt_rider', 'pcnt_car', 'pcnt_truck',
    'pcnt_bus', 'pcnt_train', 'pcnt_motorcycle', 'pcnt_bicycle', 'pcnt_unlabeled']
num_class = len(class_names)


In [18]:
# quick check

file = output_files[0]

print(os.path.basename(file).split('_pred_label.png')[0])

pred_label_img = cv2.imread(file, -1)
# count how many pixels in pred_label_img which are of object class trainId:
class_to_count = np.zeros(num_class)
for cate_id in range(num_class):
    class_to_count[cate_id] = np.sum(np.equal(pred_label_img, cate_id))/pred_label_img.size

print(class_to_count.tolist())

street2
[0.14567489801864802, 0.06896124708624708, 0.434604458041958, 0.0, 0.01633158508158508, 0.037807765151515155, 0.001757357226107226, 0.011531177156177156, 0.2095807837995338, 0.0036276223776223774, 0.02861487470862471, 0.014213650932400932, 0.000628277972027972, 0.001065340909090909, 0.00044434731934731936, 0.0, 0.0, 0.00030048076923076925, 0.002576850233100233, 0.022279283216783217]


In [20]:
#save category ratios to csv file
with open(val_dataset.output_dir + '/00_object_ratios.csv', 'w') as output:
    writer = csv.writer(output)
    writer.writerow(['image_name'] + class_names)
    
    for file in output_files:
        img_name = os.path.basename(file).split('_pred_label.png')[0]
        pred_label_img = cv2.imread(file, -1)
        
        # count how many pixels in pred_label_img which are of object class trainId:
        class_to_count = np.zeros(num_class)
        for cate_id in range(num_class):
            class_to_count[cate_id] = np.sum(np.equal(pred_label_img, cate_id))/pred_label_img.size
        
        # write to the csv file
        writer.writerow([img_name] + class_to_count.tolist())
